In [2]:
cd ..

/Users/amiyaguchi/wikipedia-retention


In [3]:
import pandas as pd

rolx_vec = pd.read_csv(
    "data/processed/user-network-v3-v", 
    header=None,
    sep=' '
)
rolx_vec.head()

,0,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,61
0,37304.0,10209240.0,15737186.0,36118260.0,968.214111,7.117394e+09,1.907944e+05,1.142605e+11,3062955.0,3443599.0,...,2.047179e+09,1216278.250,2.122445e+10,6621282.0,9047967.0,1223658.500,2.100211e+10,6632294.5,1221359.750,6624842.5
1,3546.0,720717.0,10319353.0,11757241.0,3315.634766,2.740751e+09,7.729135e+05,2.185514e+10,6163323.0,7705844.5,...,2.029646e+09,1223429.500,2.112501e+10,6613167.0,9054072.0,1224354.250,2.107670e+10,6618573.5,1223415.875,6619284.0
2,2315.0,553232.0,8424956.0,9529105.0,4116.244141,2.464032e+09,1.064377e+06,1.700661e+10,7346266.0,9470891.0,...,2.045110e+09,1217417.750,2.100859e+10,6648297.0,9077289.0,1219390.375,2.104158e+10,6638263.0,1220817.750,6632523.0
3,95.0,1712.0,1006831.0,1010160.0,10633.262695,2.003104e+08,2.108530e+06,8.012986e+08,8434722.0,12641187.0,...,2.039572e+09,1220639.500,2.094408e+10,6651448.0,9086844.0,1218765.000,2.108085e+10,6635371.0,1221411.000,6632723.5
4,451.0,26471.0,2999391.0,3051882.0,6766.922363,6.881007e+08,1.525722e+06,3.480699e+09,7717736.0,10762418.0,...,2.037694e+09,1220722.375,2.099337e+10,6641782.5,9077333.0,1220178.750,2.107218e+10,6631758.0,1221793.750,6629538.0


In [5]:
rolx_mapping = pd.read_csv(
    "data/processed/user-network-v3-mappings",
    header=None,
    skiprows=1,
    sep=' '
)

rolx_mapping.head()

,0,1
0,0,7580
1,1,2164770
2,2,32875
3,3,261299
4,4,94306


In [7]:
df = pd.concat([rolx_mapping, rolx_vec], axis=1).iloc[:, 1:]

In [8]:
# make sure to join with the mapping file afterwards
from sklearn.decomposition import NMF
import numpy as np

nmf = NMF(
    n_components=16, 
    solver='mu', 
    beta_loss='kullback-leibler', 
    max_iter=1000
)

X = df.values[:, 1:]
W = nmf.fit_transform(X)
H = nmf.components_

In [17]:
!ls data/interim/rolesense

art.txt       auth.txt      ecc_clust.txt hub.txt       pr.txt


In [10]:
pd.concat([df.iloc[:, 0], pd.DataFrame(W)]).to_csv("data/processed/rolx-nmf-G.csv", index=False)

In [52]:
from pyspark.sql import SparkSession, functions as F

spark = SparkSession.builder.getOrCreate()
enwiki = spark.read.parquet("data/processed/enwiki-meta-compact")
enwiki.createOrReplaceTempView("enwiki")

In [53]:
user_df = spark.sql("""
SELECT
    cast(user_id as int) as user_id,
    count(distinct article_id) as n_articles,
    count(*) as n_edits,
    log(sum(log(textdata))) as edit_count
FROM enwiki
WHERE cast(user_id as int) is not null
GROUP BY 1
""").toPandas()

In [42]:
pr = pd.read_csv(
    "data/interim/rolesense/pr.txt", 
    header=None, 
    names=["user_id", "pagerank"]
)
hub = pd.read_csv(
    "data/interim/rolesense/hub.txt", 
    header=None,
    names=["user_id", "hub"]
)

In [71]:
user_stats = (
    user_df
    .merge(pr, on="user_id")
    .merge(hub, on="user_id")
    .fillna(0)
)

user_stats.to_csv("data/processed/nodesense-v1.csv", index=False)

In [93]:
M = user_stats[user_stats.edit_count > 0].iloc[:10000, 1:].values

In [65]:
print(M.shape, W.shape, H.shape)

(10000, 5) (10000, 16) (16, 62)


In [66]:
help(NMF)

Help on class NMF in module sklearn.decomposition.nmf:

class NMF(sklearn.base.BaseEstimator, sklearn.base.TransformerMixin)
 |  NMF(n_components=None, init=None, solver='cd', beta_loss='frobenius', tol=0.0001, max_iter=200, random_state=None, alpha=0.0, l1_ratio=0.0, verbose=0, shuffle=False)
 |  
 |  Non-Negative Matrix Factorization (NMF)
 |  
 |  Find two non-negative matrices (W, H) whose product approximates the non-
 |  negative matrix X. This factorization can be used for example for
 |  dimensionality reduction, source separation or topic extraction.
 |  
 |  The objective function is::
 |  
 |      0.5 * ||X - WH||_Fro^2
 |      + alpha * l1_ratio * ||vec(W)||_1
 |      + alpha * l1_ratio * ||vec(H)||_1
 |      + 0.5 * alpha * (1 - l1_ratio) * ||W||_Fro^2
 |      + 0.5 * alpha * (1 - l1_ratio) * ||H||_Fro^2
 |  
 |  Where::
 |  
 |      ||A||_Fro^2 = \sum_{i,j} A_{ij}^2 (Frobenius norm)
 |      ||vec(A)||_1 = \sum_{i,j} abs(A_{ij}) (Elementwise L1 norm)
 |  
 |  For multiplic

In [138]:
nodesense = NMF(
    n_components=16,
    solver='mu', 
    beta_loss="kullback-leibler", 
    init='custom', 
    max_iter=1000
)

test = np.ones((5, 16))
E_t = nodesense.fit_transform(M.T,W=test, H=W.T)

In [139]:
np.power(M - W.dot(E_t.T), 2).sum()

5251821.950319212

In [141]:
E_t.shape

(5, 16)